# 핵심 논문 목록 (The Core Reading List)

선정 기준은 이론의 패러다임을 바꾸거나, 논쟁의 방향을 결정적으로 전환시킨 역사적 중요도입니다. 

1.  **Sargent, T. J., & Wallace, N. (1981).** *[Some Unpleasant Monetarist Arithmetic](https://www.alphaxiv.org/private/c2218a10-f843-43f5-89cd-fe965b0b0c94)*.
2.  **Leeper, E. M. (1991).** *[Equilibria Under 'Active' and 'Passive' Monetary and Fiscal Policies](https://www.alphaxiv.org/private/d21e477f-92da-41e9-b0ee-9d1d20418770)*.
3.  **Sims, C. A. (1994).** *A Simple Model for Study of the Determination of the Price Level and the Interaction of Monetary and Fiscal Policy*.
4.  **Bianchi, F., & Melosi, L. (2014).** *[Dormant Shocks and Fiscal Virtue](https://www.alphaxiv.org/private/33dbef6f-d876-4061-b9d1-0fa5b0680fc6)*.
5.  **Cochrane, J. H. (2022).** *[Fiscal Histories](https://www.alphaxiv.org/private/eb127b15-8956-41ec-baf3-48fe93fa0157)*.

---

## 논문 계보: 논쟁의 촉발과 발전 과정

이 논문들은 "인플레이션의 근원은 무엇인가?"라는 질문에 대해, 통화 정책만으로는 답할 수 없으며 재정 정책과의 상호작용이 핵심이라는 주장을 발전시켜온 역사입니다.

### 1단계: 논쟁의 촉발 - "불편한 진실"의 등장 (1980년대)

*   **Sargent & Wallace (1981) - [Some Unpleasant Monetarist Arithmetic](https://www.alphaxiv.org/private/c2218a10-f843-43f5-89cd-fe965b0b0c94)**

이 논문이 모든 논의의 시발점입니다. 당시 주류였던 통화주의(Monetarism)의 가정 하에서도, 만약 정부(재정 당국)가 지속적으로 재정 적자를 내기로 결정한다면, 중앙은행(통화 당국)은 결국 그 빚을 갚기 위해 돈을 찍어낼 수밖에 없다는 것을 산술적으로 증명했습니다. 즉, 중앙은행이 오늘 당장 인플레이션을 잡기 위해 통화 긴축을 해도, 결국 미래에는 더 큰 인플레이션으로 돌아온다는 '불편한 진실'을 제시한 것입니다. 이 논문은 **"통화 정책은 재정 정책으로부터 독립적일 수 없다"**는 화두를 던지며 논쟁을 촉발시켰습니다.

### 2단계: 이론적 기틀의 확립 (1990년대)

*   **Leeper (1991) - [Equilibria Under 'Active' and 'Passive' Monetary and Fiscal Policies](https://www.alphaxiv.org/private/d21e477f-92da-41e9-b0ee-9d1d20418770)**

리퍼의 이 논문은 논쟁을 분석할 수 있는 결정적인 **이론적 틀(Framework)**과 **언어(Vocabulary)**를 제공했습니다. 그는 정책을 '능동적(Active)'과 '수동적(Passive)'으로 구분했습니다.
    *   **능동적 통화/수동적 재정**: 중앙은행이 인플레이션을 적극적으로 통제하고, 정부는 빚을 갚기 위해 세금을 조정하는 전통적인 방식입니다. 이 경우 물가는 통화 정책에 의해 결정됩니다.
    *   **수동적 통화/능동적 재정**: 정부가 빚 상환에 무책임하고, 중앙은행은 이를 수동적으로 받아들입니다. 이 경우, 물가는 정부 부채를 실질적으로 상각시키기 위해 움직이게 됩니다. 즉, **물가가 재정 정책에 의해 결정됩니다.** 이것이 바로 재정 가격 수준 이론의 핵심입니다.

*   **Sims (1994) - A Simple Model for Study of the Determination of the Price Level...**

심스는 리퍼가 제시한 틀, 특히 '능동적 재정' 상황의 함의를 명확하게 보여주었습니다. 그는 정부의 예산 제약식(intertemporal budget constraint)이 결국 어떤 방식으로든 만족되어야 하며, 만약 정부가 미래에 세금을 올려 빚을 갚을 의지가 없다면(능동적 재정), 현재의 **물가 수준**이 상승하여 정부 부채의 실질 가치를 떨어뜨리는 방식으로 균형이 맞춰진다는 것을 단순하고 강력한 모델로 보여주었습니다. 이 논문은 재정 이론을 주류 거시경제학의 중요한 대안 이론으로 확고히 자리매김하게 했습니다.

### 3단계: 이론의 동학(Dynamics)과 현실성 보강 (2010년대)

*   **Bianchi & Melosi (2014) - [Dormant Shocks and Fiscal Virtue](https://www.alphaxiv.org/private/33dbef6f-d876-4061-b9d1-0fa5b0680fc6)**

이 논문은 재정 이론의 중요한 퍼즐, 즉 "왜 막대한 재정 적자가 항상 즉각적인 인플레이션으로 이어지지 않는가?"에 답을 제시했습니다. **'잠자는 충격(Dormant Shocks)'**이라는 개념을 통해, 재정 충격의 인플레이션 효과는 대중의 **기대(Expectations)**가 바뀌기 전까지는 잠복해 있을 수 있음을 보였습니다. 정부가 '재정적 미덕(Fiscal Virtue)'을 잃을 것이라는 비관론이 팽배해질 때 비로소 잠자던 인플레이션이 깨어난다는 것입니다. 이는 이론에 **'기대'와 '학습'** 이라는 동학적 측면을 더해 현실성을 크게 높였습니다.

### 4단계: 이론의 종합과 역사적 적용 (2020년대)

*   **Cochrane (2022) - [Fiscal Histories](https://www.alphaxiv.org/private/eb127b15-8956-41ec-baf3-48fe93fa0157)**

코크란의 이 논문은 앞선 모든 이론적 발전을 집대성하여, 실제 미국의 경제 역사를 재해석하는 데 적용합니다. 1970년대의 대인플레이션, 1980년대의 성공적인 인플레이션 억제, 그리고 2021년의 인플레이션 급등과 같은 주요 사건들을 '재정 이론'의 렌즈를 통해 일관되게 설명합니다. 이 논문은 복잡한 수식이 아닌, 설득력 있는 역사적 서사를 통해 재정 이론의 강력한 설명력을 보여주는, 이 논쟁의 흐름을 이해하기 위한 최고의 종합 해설서 역할을 합니다.

---

**결론적으로,** 이 다섯 편의 논문은 **Sargent & Wallace**가 던진 근본적인 문제 제기에서 시작하여, **Leeper**와 **Sims**가 이론적 뼈대를 세우고, **Bianchi & Melosi**가 동학적 현실성을 부여했으며, 마지막으로 **Cochrane**이 이를 종합하여 역사적 서사로 완성해가는, 재정 가격 수준 이론(FTPL)의 핵심적인 지적 계보를 형성합니다.

*참고: 함께 언급하신 다른 논문들(예: Bhattarai et al., 2014; Barro and Bianchi, 2024)은 이 핵심 흐름을 뒷받침하거나 특정 측면을 심화, 혹은 최신 데이터에 적용한 중요한 연구들이지만, 논쟁의 역사 자체를 바꾼 '계보의 핵심'으로 보기에는 위의 다섯 편이 더 중요하다고 판단하여 목록을 정제했습니다.*
관련한 두 논문에 대해서는 이하 참고.

#### 메커니즘을 파고들고 현실성을 더하다

- Inflation Dynamics (2014): 
  이 논문은 리퍼의 프레임워크를 이용해, 재정 정책이 주도하는 세상('능동적 재정' 상황)에서 어떤 일들이 일어나는지 구체적으로 분석합니다. 정부 부채가 많을수록 인플레이션이 높아지고, 중앙은행이 인플레이션을 잡으려고 할수록 오히려 변동성이 커지는 역설적인 결과를 심층적으로 보여줍니다.

- Dormant Shocks and Fiscal Virtue (2014) & The Dire Effects Of The Lack Of Monetary And Fiscal Coordination (2017): 
  이 두 논문은 이론에 현실성을 더합니다. 왜 정부 부채가 늘어도 인플레이션이 즉각 나타나지 않는가? 라는 질문에 **"잠자는 충격(Dormant Shocks)"**과 **대중의 기대(Expectations)**라는 개념으로 답합니다. 또한, 두 정책 당국이 서로 충돌할 때('능동적 통화/능동적 재정') 경제가 어떻게 악순환에 빠지는지를 보여주며 이론을 더욱 정교하게 만들었습니다.

% inflation_persist_program
% produces the response of inflation and output to monetary and fisal
% shocks for "Fiscal Histories" in Journal of Economic Perspectives
% John Cochrane, john.cochrane@stanford.edu
% Note: A modification of the program used for "inflation past present and
% future"; the latter has more variations.

In [ ]:
clear all 
close all

lw=1.5; % plot linewidth
fnt = 16; % plot font

# NK model parameters

In [ ]:

sig = 0.5; % NK model parameters
kap = 0.5;
bet = 1; 
thpi = 0.0; 
thx = 0.0; 
rho = 0.98; 
omeg = 0.9;
eta = 0.999; 

T = 100; %시뮬레이션 기간을 0~100(101개 시점)으로 설정
show_results=1;

tim = (0:T)'; % 시간축 벡터 tim을 만듦
pit = 0*tim; % 인플레이션 경로를 저장할 pit을 0으로 초기화

- sig: 소비 대체 탄력성(1/σ)

- kap: 필립스 곡선의 기울기(κ)

- bet: 할인 인자(β)

- thpi(θ_π), thx(θ_x): 테일러 규칙에서 인플레이션과 산출에 대한 반응계수
  
  * i_t = θ_π*π_t + θ_x*x_t + u_t
  * u_t: 통화정책 충격(예측 불가능한, 비체계적인 금리 변동)​
  
- rho: 충격의 자기상관(ρ)

- omeg: 채권에 대한 할인 또는 조정 파라미터(ω)

- eta: 통화정책 충격의 지속성(η)

# simple price stickiness

In [ ]:
if 0; % check; this gives the same answer 
    l1 = ((1+bet+sig*kap) + ((1+bet+sig*kap)^2-4*bet)^0.5)/2;
    pit2a = [0;(1/l1).^(tim(2:end)-1)]*(1-rho/l1);
    
    pt2a = cumsum(pit2a);
end;

% calculate and plot response to fiscal shock with fixed interest rate
% I retained individual plots. Below is the two panel plot used in JEP

- 논문 내용: 중앙은행이 금리를 바꾸지 않는 상황에서, 정부가 상환할 것이라고 예상되지 않는 재정 적자(헬리콥터 머니)가 발생하면 어떻게 될까요? 논문은 이것이 2021년 인플레이션과 유사한 상황이라고 설명하며, 이 경우 인플레이션은 즉각적으로 급등한 후 서서히 감소하는 지속적인 형태를 보인다고 주장합니다.

In [ ]:
%% 재정 충격 O, 이자율은 고정

% 재정 충격(s) 1단위, 통화 충격(u) 0단위
shock = [1 0]'; 

% 금리 정책 반응 없음 (thpi1 = 0)
thpi1 = 0; 

% 모델 풀이 및 결과 저장
[xt pit vt qt it rnt pt] = solveit(sig,kap, bet, thpi1, thx,eta, rho, omeg,T,shock);

% 그래프 그리기

figure; 
hold on;
plot(tim,0*tim,'-b','linewidth',2); % 이자율 (0으로 고정)
plot(tim, pit,'-rv', 'linewidth',2); % 인플레이션 경로
plot(tim, xt,'--ks','linewidth',2);

xlabel('Time');
ylabel('Percent');
axis([0 7 -0.05 0.45]);
text(2,0.3,'Inflation \pi','color','r','fontsize',18);
text(2.7, 0.12,'Output x','color','k','fontsize',18,'horizontalalignment','r')
text(1,0.02,'Interest rate i','color','b','fontsize',18);

    print -depsc2 sticky_inflation.eps;
    print -dpng sticky_inflation.png;

xtf = xt; % save for later
pitf = pit; % 재정 충격 시 인플레이션 경로 저장
itf = it; 

% calculate and plot response to monetary shock

- 논문 내용: 재정 흑자에 변화가 없는 상태에서 중앙은행이 금리를 1%p 인상하면 어떻게 될까요? 이 경우, 단기적으로는 수요가 위축되어 인플레이션이 하락하지만, 높아진 이자율은 정부의 이자 부담을 늘려 미래의 재정 상태를 악화시킵니다. FTPL에 따르면 이는 결국 더 높은 인플레이션을 유발합니다. 저자는 이를 "갈퀴를 밟는 것(stepping on a rake)"에 비유하며, 통화정책만으로는 재정 기반 인플레이션을 완전히 잡을 수 없다고 주장합니다.

In [ ]:
%% 재정흑자 고정, 이자율 1%p 인상

% 재정 충격(s) 0단위, 통화 충격(u) 1단위
shock = [0 1]';

% '체계적인 의지'는 배제, '재량적 결정'으로 금리가 인상되는 상황을 의미
thpi = 0; 
thx = 0; 

% => i_t=u_t(surprise rate hike)

% 모델 풀이 및 결과 저장
[xt pit vt qt it rnt pt] = solveit(sig,kap, bet, thpi, thx, eta, rho, omeg,T,shock);

% 그래프 그리기

figure;
hold on;
plot(tim,it,'-b','linewidth',2);
plot(tim, pit, '-rv','linewidth',2); 
plot(tim, xt,'--ks','linewidth',2);
plot(tim,0*tim,'-k');
xlabel('Time');
ylabel('Percent');
axis([0 10 -3 1.1])
text(2.7,-0.5,'Inflation \pi','color','r','fontsize',18,'horizontalalignment','r');
text(2.7, -1.5,'Output x','color','k','fontsize',18);
text(4,0.9,'Interest rate i','color','b','fontsize',18);

    print -depsc2 mpshock_inflation.eps;
    print -dpng mpshock_inflation.png;


disp('total inflation'); 
disp(pt(end));

% make simple plot, two panels, just inflation for JEP

In [ ]:
figure; 
subplot(2,1,1); 
hold on;
plot(tim,0*tim,'-bo','linewidth',lw);
plot(tim, pitf,'-rv', 'linewidth',lw);
%plot(tim, xtf,'--k', 'linewidth',lw);
xlabel('Time');
ylabel('Percent');
axis([0 7 -0.05 0.45]);
text(2,0.3,'Inflation \pi','color','r','fontsize',fnt);
text(1,0.05,'Interest rate i','color','b','fontsize',fnt);

subplot(2,1,2);
hold on;
plot(tim,it,'-bo','linewidth',lw);
plot(tim, pit, '-rv','linewidth',lw);
%plot(tim, xt,'--k', 'linewidth',lw);
plot(tim,0*tim,'-k');
xlabel('Time');
ylabel('Percent');
axis([0 7 -3 1.1])
%text(2.2,-1.5,'Inflation \pi','color','r','fontsize',fnt);without output
text(2,-1.5,'Inflation \pi','color','r','fontsize',fnt, 'horizontalalignment','r');%without output
%text(3,-1.5,'Output x','color','k','fontsize',fnt);

text(2.5,0.70,'Interest rate i','color','b','fontsize',fnt);

print -depsc2 JEP_simulation.eps;
print -dpng JEP_simulation.png;

% calculate and plot response to fiscal shock with Taylor rule 
% this shows how Taylor rules smooth inflation forward, reducing the impact
% inflation as claimed in the text

In [ ]:
if 0
    shock = [1 0]';
    thpi = 0.9;
    [xt pit vt qt it rnt pt] = solveit(sig,kap, bet, thpi, thx,eta, rho, omeg,T,shock);
    
    figure;
    hold on;
    plot(tim,it,'-b','linewidth',2);
    plot(tim, pit, '-rv','linewidth',2);
    plot(tim, pt, '--k','linewidth',2);
    plot(tim,0*tim,'-k');
    xlabel('Time');
    ylabel('Percent');
    axis([0 7 -0.1 1.1])
    text(2,0.3,'Inflation \pi','color','r','fontsize',18);
    text(3.5,0.75,'Price level p','color','k','fontsize',18);
    text(5,1,['p(\infty) = ' num2str(pt(end),'%4.2f')],'color','k','fontsize',18);
    text(1,0.15,'Interest rate i','color','b','fontsize',18);
    print -depsc2 policy_inflation.eps;
    print -dpng policy_inflation.png;
        
    disp('total inflation');
    disp(pt(end));
end;

In [ ]:
%******
function [xt pit vt qt it rnt pt] = solveit(sig,kap, bet, thpi, thx, eta, rho, omeg,T,shock);

show_results = 1; 

N = 4;
A = [...
  1 sig 0    0    0 ; 
  0 bet 0    0    0 ; 
  0 1   rho -omeg 0 ; 
  0 0   0    omeg 0 ;
  0 0   0     0   1]; 


% x pi q ui us

B = [ ...
    1+sig*thx sig*thpi 0 0 sig ; 
    -kap     1       0 0   0 ; 
    0        0       1 -1  0; 
    thx       thpi     0 1 1 ;
    0         0        0 0 eta]; 



C = [...
    0 0;
    0 0;
    1 0;
    0 0;
    0 1];

D = [...
    1  sig 0 ;
    0  bet 0 ;
    0  0   0 ;
    0  0   omeg ;
    0  0    0 ];

% Solve by eigenvalues See Fiscal Theory of the Price Level appendix for
% equations

In [ ]:
A1 = inv(A); 
F = A1*B; 
[Q L] = eig(F); 
Q1 = inv(Q); 
if show_results;
    disp('Eigenvalues');
    disp(abs(diag(L)'));
end
% produce Ef, Eb, that select forward and backward
% eigvenvalues. If L>=1 in position 1, 3, 
% produce
% 1 0 0 0 
% 0 0 1 0 ...
% for example

nf = find(abs(diag(L))>=1); % nf is the index of eigenvalues greater than one
if show_results
    disp('number of eigenvalues >=1');
    disp(size(nf,1))
end
if (size(nf,1) < size(D,2)); 
    disp('not enough eigenvalues greater than 1');
end;
Ef = zeros(size(nf,1),size(A,2));
Efstar = zeros(size(A,2),size(A,2));
for indx = 1:size(nf,1);
    Ef(indx,nf(indx))=1; 
    Efstar(nf(indx),nf(indx)) = 1; 
end;

nb = find(abs(diag(L))<1);
Eb = zeros(size(nb,1),size(A,2));
for indx = 1:size(nb,1);
    Eb(indx,nb(indx))=1;
end;

ze = Eb*Q1*A1*(C-D*inv(Ef*Q1*A1*D)*Ef*Q1*A1*C);
% how epsilon shocks map to z.
% in principle the forward z are zero. In practice they are 1E-16 and then
% grow. So I go through the trouble of simulating forward only the nonzero
% z and eigenvalues less than one.

Nb = size(Eb,1); % number of stable z's
Lb = (Eb*L*Eb'); % diagonal with only stable Ls

% impulse response

In [ ]:
H = T+1;

%shock = [ 1 0]' for fiscal [ 0 1 ]' for monetary; 

zbt = zeros(H,Nb);
zbt(2,:) = (ze*shock)';
for t = 3:H;
    zbt(t,:) = Lb*zbt(t-1,:)';
end;

% now create the full z with zeros
zt = zeros(H,N);
zt(:,nb) = zbt;

% create original variables
yt = zt*Q';

xt = yt(:,1);
pit = yt(:,2);
vt = yt(:,3);
qt = yt(:,4);
ut = yt(:,5);

rnt = zeros(H,1);
rnt = omeg*qt-[0;qt(1:H-1)];

it = thpi*pit + thx*xt + ut;

%yield is nonlinear so use the right size
qlevelt = qt - log(1-omeg);
yldt = exp(-qlevelt)+omeg-1; 

pt = cumsum(pit);
disp('omeg')
disp(omeg)
end